In [0]:
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# --- CONFIGURAÇÃO DA CONEXÃO COM POSTGRESQL (SUPABASE) ---
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Validação para garantir que as variáveis foram carregadas
if not all([db_user, db_password, db_host, db_port, db_name]):
    raise ValueError("Uma ou mais variáveis de ambiente do banco de dados não foram definidas. Verifique o arquivo .env")

jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}"

connection_properties = {
    "user": db_user,
    "password": db_password,
    "driver": "org.postgresql.Driver"
}

# --- PROCESSO DE INGESTÃO ---

# Lista de tabelas para ingestão
tables = [
    "companies", "company_reviews", "employment_types", "industries",
    "job_benefits", "job_skills", "jobs", "locations", "salary_ranges", "skills"
]

# Loop para ler cada tabela e salvar na camada Landing como CSV
for table_name in tables:
    print(f"Iniciando ingestão da tabela: {table_name}")

    try:
        # Leitura da tabela do banco de dados
        df = spark.read.jdbc(url=jdbc_url, table=f"public.{table_name}", properties=connection_properties)

        # Define o caminho de destino na camada Landing
        landing_path = f"/mnt/datalake/landing/{table_name}"

        # Salva os dados como CSV
        df.write.format("csv").option("header", "true").mode("overwrite").save(landing_path)

        print(f"Tabela {table_name} salva em {landing_path}")
    
    except Exception as e:
        print(f"ERRO ao ingerir a tabela {table_name}: {e}")
        break